In [ ]:
import numpy as np
from scipy.io import loadmat
from tqdm import tqdm
from scipy.io import savemat

In [ ]:
NYC_data_path = 'dataset/dataset_connected_NYC.mat'

In [3]:
data = loadmat(NYC_data_path)

In [44]:
selected_checkins = data['selected_checkins']

In [5]:
old_friendship = data['friendship_old']
new_friendship = data['friendship_new']
friendship = np.concatenate((old_friendship, new_friendship), axis=0)

In [6]:
friendship

array([[   1,    2],
       [   1,   24],
       [   1,   30],
       ...,
       [3959, 3965],
       [3970, 4000],
       [3981, 3991]], dtype=uint16)

In [9]:
def colocation_friendship_statistics(friends, checkins):
    def get_checkin_from_user(user, checkins):
        locations = set()
        for i in range(len(checkins)):
            if checkins[i][0] == user:
                locations.add(checkins[i][2])
        return locations
    
    def jaccard_similarity(set1, set2):
        inter = set1.intersection(set2)
        uni = set1.union(set2)
        if len(uni) == 0:
            return 0
        return len(inter) / len(uni)
    
    simis = []
    
    for i in tqdm(range(len(friends))):
        source, target = friends[i][0], friends[i][1]
        source_checkins = get_checkin_from_user(source, checkins)
        target_checkins = get_checkin_from_user(target, checkins)        
        simis.append(jaccard_similarity(source_checkins, target_checkins))
    return np.mean(simis)

In [10]:
colocation_friendship_statistics(friendship, selected_checkins)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19268/19268 [56:47<00:00,  5.65it/s]


0.02626099581260058

In [11]:
map_dict = 'Suhi_output/ego_net_NYC.txt'
edge_list = 'Suhi_output/edgelist_NYC'

In [12]:
maps = dict()
with open(map_dict, 'r', encoding='utf-8') as file:
    for line in file:
        data_line = line.strip().split(',')
        maps[data_line[0]] = data_line[1]


In [45]:
user_checkins_dict = dict()
for i in range(len(selected_checkins)):
    user = selected_checkins[i][0]
    venue = selected_checkins[i][2]
    if user not in user_checkins_dict:
        user_checkins_dict[user] = set([venue])
    else:
        user_checkins_dict[user].add(venue)

In [46]:
len(user_checkins_dict.keys())

3785

In [47]:
max(list(user_checkins_dict))

4024

In [15]:
import networkx as nx
ego_net =nx.read_edgelist(edge_list)

In [48]:
def get_checkin_from_user(user, checkins):
    try:
        return user_checkins_dict[int(user)]
    except:
#         print(user)
        return set()

def jaccard_similarity(set1, set2):
    inter = set1.intersection(set2)
    uni = set1.union(set2)
    if len(uni) == 0:
        return 0
    return len(inter) / len(uni)


def colocation_friendship_statistics_for_ego(ego_net, checkins, maps):
    scores = []
    count = 0
    for node in tqdm(ego_net.nodes()):
        neighbors = ego_net.neighbors(node)
#         print(neighbors)
        if len(neighbors) > 1:
            for i in range(len(neighbors) - 1):
                for j in range(i + 1, len(neighbors)):
                    if ego_net.has_edge(neighbors[i], neighbors[j]):
                        node_i = int(maps[neighbors[i]])
                        node_j = int(maps[neighbors[j]])
                        location_i = get_checkin_from_user(node_i, checkins)
                        location_j = get_checkin_from_user(node_j, checkins)
                        score = jaccard_similarity(location_i, location_j)
                        scores.append(score)
        count += 1
        
    return np.mean(scores)

In [49]:
colocation_friendship_statistics_for_ego(ego_net, selected_checkins, maps)




  0%|                                                                                                                                                              | 0/11412 [00:00<?, ?it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11412/11412 [00:00<00:00, 70632.73it/s]


0.026604663944231395

In [2]:
friendship_old = np.load('CA Dataset/old_friendship_new.npy')
friendship_new = np.load('CA Dataset/new_friendship_new.npy')
selected_checkins = np.load('CA Dataset/selected_checkins_new.npy')

dictt = {'friendship_old': friendship_old, 'friendship_new': friendship_new, 'selected_checkins': selected_checkins}

In [3]:
savemat('dataset/dataset_connected_hongzhi.mat', dictt)